In [1]:
# Importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import pandas as pd 
from data import L_D_Hours, Wind_Hours, PC_DA, P_max, Load_Demand, Cost_Generation, Wind_Farm_Power, Wind, T, N, P_min # Import data from data.py

h = 7  # Considering the 7th hour in the dataset 

# Defining the number of demand nodes and wind farms 
Dem = len(L_D_Hours)  # Number of demand nodes
Wind = len(Wind_Hours)  # Number of wind farms
Gen = len(P_max)  # Number of generators

# Loading bid data from CSV
file_path = "day_ahead_bids.csv"
df = pd.read_csv(file_path)
df.columns = ["Hour", "Bid Price", "Bid Amount"]

# Convert bid data into a dictionary grouped by hour
bid_data = {}
for hour, group in df.groupby("Hour"):
    bid_data[hour] = group[["Bid Price", "Bid Amount"]].to_dict(orient="records")

# Create a bid price map for each demand node (1 to Dem) for each hour
bid_prices = {}  
for hour in range(1, T + 1):
    # Get the sorted bids by price for the current hour
    sorted_bids = sorted(bid_data.get(hour, []), key=lambda x: x["Bid Price"])
    
    # Allocate bid prices to demand nodes based on sorted bids
    total_bid_amount = 0
    bid_prices[hour] = []
    
    for bid in sorted_bids:
        bid_price = bid["Bid Price"]
        bid_amount = bid["Bid Amount"]
        
        # Allocate the bid price to demand nodes
        for _ in range(int(bid_amount)):  
            bid_prices[hour].append(bid_price)
    
    if len(bid_prices[hour]) < Dem:
        # Fill the remaining demand nodes with the highest available price
        highest_price = sorted_bids[-1]["Bid Price"]
        bid_prices[hour].extend([highest_price] * (Dem - len(bid_prices[hour])))

# Model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Load and wind profile for hour h
Load = np.zeros(Dem)
Load[:] = L_D_Hours[:, h]     # load demand at hour h
Wind_profile = np.zeros(Wind)
Wind_profile[:] = Wind_Hours[:, h]  # Wind production at hour h

# Decision Variables
model.P_g = Var(range(Gen), domain=NonNegativeReals)  # Production of conventional generators
model.P_w = Var(range(Wind), domain=NonNegativeReals)  # Production of wind farms
model.P_d = Var(range(Dem), domain=NonNegativeReals)  # Demand

# Objective function: maximising social welfare
def objective_rule(model):
    return sum(bid_prices[h][dem] * model.P_d[dem] for dem in range(Dem)) - \
           sum(PC_DA[gen] * model.P_g[gen] for gen in range(Gen)) - \
           sum(0 * model.P_w[wind] for wind in range(Wind)) # No cost for wind production

model.obj = Objective(rule=objective_rule, sense=maximize)

# Constraints

# Capacity constraints for generators
def conventional_generation_rule(model, gen):
    return model.P_g[gen] <= P_max[gen]

model.conventional_generation = Constraint(range(Gen), rule=conventional_generation_rule)

# Capacity constraints for wind 
def wind_generation_rule(model, wind):
    return model.P_w[wind] <= Wind_profile[wind]  # Wind generation cannot exceed the profile

model.wind_generation = Constraint(range(Wind), rule=wind_generation_rule)

# Demand constraint
def demand_rule(model, dem):
    return model.P_d[dem] <= L_D_Hours[dem, h]  # Use the correct hour's demand

model.demand = Constraint(range(Dem), rule=demand_rule)

# Balance constraint (Total demand = total generation)
def balance_rule(model):
    return sum(model.P_d[dem] for dem in range(Dem)) == \
           sum(model.P_g[gen] for gen in range(Gen)) + \
           sum(model.P_w[wind] for wind in range(Wind))

model.balance = Constraint(rule=balance_rule)

# Solve the model using GLPK and ensuring dual values are available
solver = SolverFactory('glpk')
solver.solve(model)

# Solution
if model.obj() is not None:
    print(f"Optimal objective value (Social Welfare): {model.obj()} ")
    print("")

    # Production of wind farms
    for wind in range(Wind):
        print(f"Production of Wind farm # {wind}: {model.P_w[wind].value} (MW)")
    print("")

    # Production of generators
    for gen in range(Gen):
        print(f"Production of Conventional generator # {gen}: {model.P_g[gen].value} (MW)")
    print("")

    # Market Clearing Price
    MC_price = model.dual[model.balance]  # Accessing dual value for the balance constraint
    print(f"Market clearing price: {MC_price} ($/MWh)")
    print("")

    # Profit for Generators
    Revenue_p_g = [int(MC_price * model.P_g[gen].value) for gen in range(Gen)]
    Cost_p_g = [int(PC_DA[gen] * model.P_g[gen].value) for gen in range(Gen)]
    Profit_p_g = [Revenue_p_g[gen] - Cost_p_g[gen] for gen in range(Gen)]
    
    for gen in range(Gen):
        print(f"Total Profit of Conv. unit # {gen}: ${Profit_p_g[gen]}")
    print("")

    # Profit for Wind farms
    Revenue_p_wf = [int(MC_price * model.P_w[wind].value) for wind in range(Wind)]
    Cost_p_wf = [int(0 * model.P_w[wind].value) for wind in range(Wind)] 
    Profit_p_wf = [Revenue_p_wf[wind] - Cost_p_wf[wind] for wind in range(Wind)]
    
    for wind in range(Wind):
        print(f"Total Profit of Wind Farm # {wind}: ${Profit_p_wf[wind]}")
    print("")

    # Profit for Demand
    Revenue_Demand = [int(model.P_d[dem].value * bid_prices[h][dem]) for dem in range(Dem)]
    Cost_Demand = [int(model.P_d[dem].value * MC_price) for dem in range(Dem)]
    Profit_Demand = [Revenue_Demand[dem] - Cost_Demand[dem] for dem in range(Dem)]
    
    for dem in range(Dem):
        print(f"Total Profit of Demand #{dem}: ${Profit_Demand[dem]}")
    print("")

    print("Total costs and profit:")
    print(f"Total Revenue G: ${sum(Revenue_p_g)}")
    print(f"Total Cost G: ${sum(Cost_p_g)}")
    print(f"Total Profit G: ${sum(Profit_p_g)}")
    print(f"Total Revenue WF: ${sum(Revenue_p_wf)}")
    print(f"Total Cost WF: ${sum(Cost_p_wf)}")
    print(f"Total Profit WF: ${sum(Profit_p_wf)}")
    print(f"Total Revenue Demand: ${sum(Revenue_Demand)}")
    print(f"Total Cost Demand: ${sum(Cost_Demand)}")
    print(f"Total Profit Demand: ${sum(Profit_Demand)}")
    print("")

else:
    print("No optimal solution available")


Optimal objective value (Social Welfare): 81112.04042428802 

Production of Wind farm # 0: 143.1758086 (MW)
Production of Wind farm # 1: 146.418634 (MW)
Production of Wind farm # 2: 87.212025 (MW)
Production of Wind farm # 3: 121.150513 (MW)
Production of Wind farm # 4: 127.118818 (MW)
Production of Wind farm # 5: 103.6143558 (MW)

Production of Conventional generator # 0: 0.0 (MW)
Production of Conventional generator # 1: 0.0 (MW)
Production of Conventional generator # 2: 0.0 (MW)
Production of Conventional generator # 3: 0.0 (MW)
Production of Conventional generator # 4: 0.0 (MW)
Production of Conventional generator # 5: 155.0 (MW)
Production of Conventional generator # 6: 155.0 (MW)
Production of Conventional generator # 7: 400.0 (MW)
Production of Conventional generator # 8: 400.0 (MW)
Production of Conventional generator # 9: 300.0 (MW)
Production of Conventional generator # 10: 140.8098456 (MW)
Production of Conventional generator # 11: 0.0 (MW)

Market clearing price: 10.52 ($/M